<a href="https://colab.research.google.com/github/fininsight/text-mining-tutorial/blob/master/4_%EB%AC%B8%EC%84%9C%EC%9A%94%EC%95%BD_Text_Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 문서 요약 (Text Summarization)

In [0]:
#https://www.businessinsider.sg/faang-stocks-are-a-dead-trade-for-the-next-several-months-2018-3/amp/

Text = "The FAANG stocks won’t see much more growth in the near future, according to Bill Studebaker, founder and Chief Investment Officer of Robo Global. \
Studebaker argues we are seeing a 'reallocation' that will continue from large-cap tech stocks into market-weight stocks. \
The FAANG stocks have had a rough few weeks, and have been hit hard since March 12. \
One FAANG to look out for, in the midst of all this, is Amazon, according to Studebaker. \
The stock market is seeing a 'reallocation' out of FAANG stocks, which are not where the smart money is, founder and Chief Investment Officer of Robo Global Bill Studebaker told Business Insider. \
The FAANG stocks (Facebook, Apple, Amazon, Netflix, Google) are all down considerably since March 12, a trend that accelerated when news of a massive Facebook data scandal broke, sending the tech-heavy Nasdaq into a downward frenzy. \
Investors are wondering what’s next. \
And what’s next isn’t good news for FAANG stock optimists, Studebaker thinks. 'This is a dead trade' for the next several months, he said. 'I wouldn’t expect there to be a lot of performance attribution coming from the FAANG stocks,' he added. That is, if the stock market is to see gains in the next several months, they will largely not come from the big tech companies. \
The market is seeing a 'reallocation out of large-cap technology, into other parts of the market,' he said. And this trend could continue for the foreseeable future. 'When you get these reallocation trades, a de-risking, this can go on for months and months.' The FAANG’s are pricey stocks, he said, pointing out that investors will 'factor in the law of big numbers,' he said. 'Just because they’re big cap doesn’t mean they’re safe,' he added. \
Still, he doesn’t necessarily think that investors are going to shift drastically into value stocks. 'With an increasingly favorable macro backdrop, you have strong growth demand.' \
Studebaker, who runs an artificial intelligence and robotics exchange-traded fund with $4 billion in assets under management, thinks that AI and robotics are better areas of growth. His ETF is up 27% in the past year, while the FAANG stocks are also largely up over that same span, even if they are down since March 12. \
While many point to artificial intelligence as an area that will be a boost to Google and Amazon, Studebaker doesn’t see that as a sign of significant growth for the FAANGs. He pointed out that 'eighty to ninety percent of their businesses are still search,' and that 'AI doesn’t really move the needle on the business.' He also said 'the revenue mix [attributable to AI] in those businesses are insignificant.' \
And while he’s not bullish on FAANG’s, he does say that the one FAANG to still watch out for is Amazon, simply because ecommerce still represents a small portion of the global retail market, giving the company room to grow." 

# 1 Textrank

![대체 텍스트](https://www.researchgate.net/profile/Khushboo_Thakkar3/publication/232645575/figure/fig1/AS:575720050573312@1514273764062/Sample-graph-build-for-sentence-extraction.png)

## 1.1 TextRank 직접 구현하기 (Matrix 활용)

In [27]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
import numpy as np
import re
from operator import itemgetter

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [28]:
# 코사인 유사도 (1. 단어의 표현 예제 참고)
def cosine_similarity(x, y):
    # x와 y, 두 벡터의 코사인 유사도를 계산하는 함수
    nominator = np.dot(x, y)    # 분자
    denominator = np.linalg.norm(x)*np.linalg.norm(y)  # 분모
    return nominator/denominator
  
cosine_similarity([0,0,1,0],[0,0,1,1])  

0.7071067811865475

In [29]:
# 문장간 유사도 측정 (BoW를 활용 코사인 유사도 측정)
def sentence_similarity(sentence1, sentence2):
    # 각 문장을 소문자로 변환
    sentence1 = [word.lower() for word in sentence1.split()]
    sentence2 = [word.lower() for word in sentence2.split()]
    
    # BoW 생성을 위한 unique한 단어로 배열 생성
    words_ls = list(set(sentence1 + sentence2))
 
    bow1 = [0] * len(words_ls)
    bow2 = [0] * len(words_ls)
 
    # 첫번째 문장 BoW 생성
    for word in sentence1:
        bow1[words_ls.index(word)] += 1

    # 두번째 문장 BoW 생성
    for word in sentence2:
        bow2[words_ls.index(word)] += 1
 
    return cosine_similarity(bow1, bow2)

sentence_similarity('나는 치킨을 좋아해','나는 치킨을 싫어해')

0.6666666666666667

In [46]:
 def buildMatrix(sentences):
    # 문장별로 그래프 edge를 Matrix 형태로 생성
    weighted_edge = np.zeros((len(sentences), len(sentences)),dtype=np.float32)
    
    #print(len(weighted_edge))
    #print(len(sentences))

    for i in range(len(sentences)):
        for j in range(len(sentences)):
            if i == j:
                continue
            weighted_edge[i][j] = sentence_similarity(sentences[i], sentences[j])
            #print('[' + str(i) + ',' + str(j)+'] = ' + str(weighted_edge[i][j]))
 
    # normalize 
    for i in range(len(weighted_edge)):
        weighted_edge[i] /= weighted_edge[i].sum()
 
    return weighted_edge
 
S = buildMatrix(sent_tokenize(Text))    
print(S[:3])

[[0.         0.0112262  0.05309461 0.08495137 0.10800254 0.04884599
  0.         0.0267237  0.0267237  0.05454953 0.0748202  0.05921028
  0.04628681 0.0115717  0.05554416 0.         0.0119512  0.013956
  0.04156678 0.05476728 0.05929456 0.06016671 0.02474134 0.07200538]
 [0.01794262 0.         0.04197839 0.         0.07624147 0.09654821
  0.04091547 0.02641082 0.02641082 0.04312869 0.06572819 0.07802273
  0.03049659 0.02287244 0.05489386 0.         0.09449022 0.
  0.03286409 0.06185837 0.08790053 0.05096767 0.02445167 0.02587721]
 [0.07593698 0.03756431 0.         0.03553229 0.06453402 0.1050718
  0.         0.04471048 0.04471048 0.05475893 0.04172635 0.04953131
  0.05162721 0.03872041 0.03097633 0.         0.         0.04669857
  0.02781756 0.09162921 0.04960183 0.0431412  0.         0.06571072]]


In [0]:
def scoring(A, eps=0.0001, d=0.85, max_iter = 50):
    P = np.ones(len(A)) / len(A)
    for iter in range(0,max_iter):
        #print(iter)
        newP = np.ones(len(A)) * (1 - d) / len(A) + d * A.T.dot(P)
        
        if abs((newP - P).sum()) <= eps:
            return newP
        P = newP


In [32]:
a = scoring(S)
a

array([0.05131312, 0.03476392, 0.04586214, 0.04100753, 0.05375078,
       0.04911632, 0.02083343, 0.02928484, 0.04381788, 0.04545217,
       0.04992899, 0.05063568, 0.03620586, 0.02920778, 0.06010798,
       0.01512391, 0.04080827, 0.01164713, 0.03514574, 0.04637666,
       0.06060599, 0.05939736, 0.03014937, 0.05945716])

In [0]:
def summarize(text, linesinSummary=10):
  text = sent_tokenize(text)
  weighted_edge = buildMatrix(text) 
  score = scoring(weighted_edge)
  #print(score) 

  rankedSentenceIndexes = [item[0] for item in sorted(enumerate(score), key=lambda item: -item[1])]
  selectedSentences = sorted(rankedSentenceIndexes[:linesinSummary])
  summary = itemgetter(*selectedSentences)(text)
  return summary

In [40]:
summarize(Text,3)

("The FAANG’s are pricey stocks, he said, pointing out that investors will 'factor in the law of big numbers,' he said.",
 'While many point to artificial intelligence as an area that will be a boost to Google and Amazon, Studebaker doesn’t see that as a sign of significant growth for the FAANGs.',
 'And while he’s not bullish on FAANG’s, he does say that the one FAANG to still watch out for is Amazon, simply because ecommerce still represents a small portion of the global retail market, giving the company room to grow.')



---



## 1.2 TextRank 직접 구현하기 (Graph 활용)

In [47]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
import networkx as nx
import math
import matplotlib.pyplot as plt
def sentences(text):
    return sent_tokenize(text)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
def connect(nodes):
    return [(start,end ,sentence_similarity(start, end)) for start in nodes for end in nodes if start is not end]

In [0]:
#def similarity(c1,c2):
#    return len(common_words(c1, c2))/(math.log(len(c1))+math.log(len(c2)))

In [0]:
#def common_words(c1,c2):
#    elem1 = [x for x in c1.split()]
#    elem2 = [x for x in c2.split()]
#    c3=list()
#    for item in elem1:
#        if item in elem2:
#            c3.append(item)
#    return c3

In [0]:
def rank(nodes,edges):
    graph=nx.diamond_graph()
    graph.add_nodes_from(nodes)
    graph.add_weighted_edges_from(edges)
    return nx.pagerank(graph)

In [0]:
def summarize(text,num_summaries=6):
    nodes=sentences(text)
    edges=connect(nodes)
    scores=rank(nodes,edges)
    #print(nodes)
    return sorted(scores,key=scores.get)[:num_summaries]


In [53]:
summary=summarize(Text, 3)

for sent in summary :
  print(sent)

['The FAANG stocks won’t see much more growth in the near future, according to Bill Studebaker, founder and Chief Investment Officer of Robo Global.', "Studebaker argues we are seeing a 'reallocation' that will continue from large-cap tech stocks into market-weight stocks.", 'The FAANG stocks have had a rough few weeks, and have been hit hard since March 12.', 'One FAANG to look out for, in the midst of all this, is Amazon, according to Studebaker.', "The stock market is seeing a 'reallocation' out of FAANG stocks, which are not where the smart money is, founder and Chief Investment Officer of Robo Global Bill Studebaker told Business Insider.", 'The FAANG stocks (Facebook, Apple, Amazon, Netflix, Google) are all down considerably since March 12, a trend that accelerated when news of a massive Facebook data scandal broke, sending the tech-heavy Nasdaq into a downward frenzy.', 'Investors are wondering what’s next.', 'And what’s next isn’t good news for FAANG stock optimists, Studebaker

## 1.2 gensim을 활용한 요약

In [12]:
from gensim.summarization.summarizer import summarize
print(summarize(Text))

The FAANG stocks won’t see much more growth in the near future, according to Bill Studebaker, founder and Chief Investment Officer of Robo Global.
Studebaker argues we are seeing a 'reallocation' that will continue from large-cap tech stocks into market-weight stocks.
The stock market is seeing a 'reallocation' out of FAANG stocks, which are not where the smart money is, founder and Chief Investment Officer of Robo Global Bill Studebaker told Business Insider.




---



#  2 Luhn Summarizer

In [0]:
import pandas as pd
from konlpy.tag import Okt
okt = Okt()

from sklearn.feature_extraction.text import TfidfVectorizer

In [0]:
df1 = pd.read_csv('preprocessing_실리콘벨리기사.csv')

In [0]:
df1.rename(columns={'0':'raw','1':'tuned'},inplace=True)

In [0]:
print(df1['raw'].tolist())

['교육을 삶의 최우선 순위로 두고 있는 한국의 부모들은 대학 전공 가운데 의학과 공학·과학을 중시한다. 자녀의 직업적 성공을 위해 대학 전공으로 의학과 이공계를 우선적으로 고려하는 일은 한국이 산업화 중이던 상황에선 올바른 선택이었다. 하지만 지금은 모든 것이 달라졌다. 요즘 실리콘밸리에서 확인되는 것은 4차 산업혁명 시대에는 예술과 인문학이 의학·공학만큼 중요하다는 사실이다.스티브 잡스는 자신이 대학 시절 수강했던 서체(書體) 수업이 매킨토시(애플이 1984년 발표한 개인용 컴퓨터) 개발 성공에 큰 영향을 미쳤다고 말했다. 그는 2011년 아이패드 2를 공개하면서 "애플의 DNA는 기술만으로는 충분하지 않다. 교양과 인문학이 결합한 기술이야말로 가슴 벅찬 결과를 낳을 것"이라며 예술과 디자인의 중요성을 강조했다. 이런 관점을 바탕으로 잡스는 세계 최고 가치를 인정받는 기업을 만들었고, 기술 산업의 새로운 표준까지 정했다.실리콘밸리에서 최근 뜨고 있는 스타 기업인 중에는 인문학 전공자들이 제법 많다. 구인·구직 소셜 네트워킹 서비스 기업인 링크드인(LinkedIn) 창업자 리드 호프만은 철학 석사 학위 소지자이며, 수잔 보이치키 유튜브 CEO는 역사와 문학을 전공했다. 메신저 개발 업체 슬랙(Slack)의 창업자 스튜어트 버터필드는 철학, 세계 최대 숙박 공유 기업인 에어비앤비의 설립자 브라이언 체스키는 미술을 전공했다. 중국 알리바바그룹의 마윈 회장의 학부 전공은 영어였다.내가 속해 있는 하버드대·듀크대 연구팀은 미국 IT 기업 창업자들의 92%가 학사 학위를, 47%는 석사 학위 이상을 갖고 있음을 밝혀냈다. 창업자들의 세부 전공을 보면 37%만 공학·컴퓨터 기술이며, 수학 전공자는 2%뿐이었다. 이들의 전공은 경영·회계·보건·예술·인문학 등 매우 다양했다.컴퓨터 주변기기 제조 업체 로지텍의 브랙큰 대럴 CEO도 영문학을 전공했다. 최근 내가 그에게 "어떻게 5년여 만에 회사 주가를 450% 올릴 수 있었느냐"고 물었더니 그는 "우리 회사가 만드는 모든 

In [0]:
doc = df1['raw'].tolist()
sentences = doc[0].split(".")
sentences

['교육을 삶의 최우선 순위로 두고 있는 한국의 부모들은 대학 전공 가운데 의학과 공학·과학을 중시한다',
 ' 자녀의 직업적 성공을 위해 대학 전공으로 의학과 이공계를 우선적으로 고려하는 일은 한국이 산업화 중이던 상황에선 올바른 선택이었다',
 ' 하지만 지금은 모든 것이 달라졌다',
 ' 요즘 실리콘밸리에서 확인되는 것은 4차 산업혁명 시대에는 예술과 인문학이 의학·공학만큼 중요하다는 사실이다',
 '스티브 잡스는 자신이 대학 시절 수강했던 서체(書體) 수업이 매킨토시(애플이 1984년 발표한 개인용 컴퓨터) 개발 성공에 큰 영향을 미쳤다고 말했다',
 ' 그는 2011년 아이패드 2를 공개하면서 "애플의 DNA는 기술만으로는 충분하지 않다',
 ' 교양과 인문학이 결합한 기술이야말로 가슴 벅찬 결과를 낳을 것"이라며 예술과 디자인의 중요성을 강조했다',
 ' 이런 관점을 바탕으로 잡스는 세계 최고 가치를 인정받는 기업을 만들었고, 기술 산업의 새로운 표준까지 정했다',
 '실리콘밸리에서 최근 뜨고 있는 스타 기업인 중에는 인문학 전공자들이 제법 많다',
 ' 구인·구직 소셜 네트워킹 서비스 기업인 링크드인(LinkedIn) 창업자 리드 호프만은 철학 석사 학위 소지자이며, 수잔 보이치키 유튜브 CEO는 역사와 문학을 전공했다',
 ' 메신저 개발 업체 슬랙(Slack)의 창업자 스튜어트 버터필드는 철학, 세계 최대 숙박 공유 기업인 에어비앤비의 설립자 브라이언 체스키는 미술을 전공했다',
 ' 중국 알리바바그룹의 마윈 회장의 학부 전공은 영어였다',
 '내가 속해 있는 하버드대·듀크대 연구팀은 미국 IT 기업 창업자들의 92%가 학사 학위를, 47%는 석사 학위 이상을 갖고 있음을 밝혀냈다',
 ' 창업자들의 세부 전공을 보면 37%만 공학·컴퓨터 기술이며, 수학 전공자는 2%뿐이었다',
 ' 이들의 전공은 경영·회계·보건·예술·인문학 등 매우 다양했다',
 '컴퓨터 주변기기 제조 업체 로지텍의 브랙큰 대럴 CEO도 영문학을 전공했다',
 ' 최근 내가

In [0]:
tuned_sentences = []

for idx in range(len(sentences)) : 
    tuned_sentences.append(okt.nouns(sentences[idx]))
tuned_sentences[0]

['교육',
 '삶',
 '최',
 '우선',
 '순위',
 '한국',
 '부모',
 '대학',
 '전공',
 '가운데',
 '의학과',
 '공학',
 '과학',
 '중시']

In [0]:
tuned_sentences = [','.join(tuned_sentences[idx]).replace(',',' ') for idx in range(len(tuned_sentences))]
tuned_sentences

['교육 삶 최 우선 순위 한국 부모 대학 전공 가운데 의학과 공학 과학 중시',
 '자녀 직업 성공 위해 대학 전공 의학과 이공 우선 고려 일 한국 산업화 중 상황 선택',
 '지금 모든 것',
 '요즘 실리콘밸리 확인 것 차 산업혁명 시대 예술 인문학 의학 공학 사실',
 '스티브 잡스 자신 대학 시절 수강 서체 수업 매킨토시 애플 발표 개인 용 컴퓨터 개발 성공 영향 말',
 '그 아이패드 를 공개 애플 기술',
 '교양 인문학 결합 기술 가슴 결과 낳을 것 라며 예술 디자인 중요성 강조',
 '관점 바탕 잡스 세계 최고 가치 기업 기술 산업 표준 정',
 '실리콘밸리 최근 스타 기업인 중 인문학 전공자 제법',
 '구인 구직 소셜 네트워킹 서비스 기업인 링크드인 창업 리드 호프 철학 석사 학위 소지 잔 보이 유튜브 역사 문학 전공',
 '메신저 개발 업체 슬랙 의 창업 스튜어트 버터 필드 철학 세계 최대 숙박 공유 기업인 에어비앤비 설립 브라이언 체 스키 미술 전공',
 '중국 알리바바 그룹 마윈 회장 학부 전공 영어',
 '내 해 하버드대 듀크 팀 미국 기업 창업 자 학사 학위 석사 학위 이상 음',
 '창업 자 세부 전공 만 공학 컴퓨터 기술 수학 전공자',
 '이 전공 경영 회계 보건 예술 인문학 등 매우',
 '컴퓨터 주변기기 제조 업체 로지텍 브랙큰 대럴 도 영문학 전공',
 '최근 내 그 회사 주가 수 고 그 우리 회사 모든 제품 디자인 고민 때문',
 '기술 관련 제품 성공 가장 것 디자인 답',
 '차 산업혁명 혁신 규칙 일이',
 '여기 컴퓨터 인공 지능 디지털 의술 로봇공학 합성생물학 등 기술 활용',
 '의학 인공 지능 센서 융합 인간 건강 진단 질병 예방 디지털 의사 를 수',
 '유전체학 유전자 편집 이용 가뭄 인류 전체 새 식물 개발 수',
 '인공 지능 로봇 사용 노인 위 디지털 친구 도',
 '초소 물질 발전 모든 사람 이용 수 태양열 저장 기술 의 새 시대 열 것',
 '해결 책 데 생물학 교육 의료 인간 행동 등 여러 분

In [0]:
vectorizer = TfidfVectorizer()
tfidf_vector = vectorizer.fit_transform(tuned_sentences).toarray()

In [0]:
luhn_importance = []

for idx in range(len(tfidf_vector)) : 
    luhn_importance.append(sum(tfidf_vector[idx]) / len([i for i in tfidf_vector[idx] if i != 0]))
luhn_importance[:5]

/home/hskimim/anaconda3/envs/dss_env/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in double_scalars
  after removing the cwd from sys.path.


[0.2864330566455741,
 0.2655826626486411,
 0.703848815311294,
 0.3138733801351376,
 0.24904542659640516]

In [0]:
final_df = pd.DataFrame(columns=['sentence','luhn_importance'])
final_df['sentence'] = sentences
final_df['luhn_importance'] = luhn_importance
final_df.sort_values('luhn_importance',ascending=False).head()

,sentence,luhn_importance
2,하지만 지금은 모든 것이 달라졌다,0.703849
18,4차 산업혁명은 '혁신의 규칙'을 바꾸는 일이다,0.498424
5,"그는 2011년 아이패드 2를 공개하면서 ""애플의 DNA는 기술만으로는 충분하지 않다",0.489840
33,그 대신 자녀가 자신의 열정을 추구하고 배우는 걸 즐길 수 있도록 북돋아야 한다,0.445919
26,이는 인문학 전공자들이 가장 잘 훈련받은 분야이다,0.444194
